In [23]:
import os, glob, yaml
from collections import ChainMap
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


def get_metric_files(logs_root):
    version_roots = glob.glob(os.path.join(logs_root, '*'))
    version_data = []
    for version_root in version_roots:
        metric_file = os.path.join(version_root, 'metrics.csv')
        metadata_file = os.path.join(version_root, 'metadata.yaml')
        # checkpoint_file = os.path.join(version_root, glob.glob(os.path.join(version_root, 'checkpoints', '*'))[0])
        checkpoint_file = None
        if os.path.exists(metric_file):
            version_data.append((metric_file, metadata_file,  checkpoint_file))
    return version_data

def get_metadata(metadata_file):
    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as f:
            raw = yaml.safe_load(f)
            metadata = dict(ChainMap(*raw.get('model', [])))
            for key in ('time', 'benchmark'):
                if key in metadata and isinstance(metadata[key], list):
                    metadata[key] = dict(ChainMap(*metadata[key]))
        return metadata
    else:
        return None
    
def get_metrics(metrics_file):
    if os.path.exists(metrics_file):
        return pd.read_csv(metrics_file)
    else:
        return None

In [24]:
def df_to_latex_table(df, short_caption, long_caption, label):
    from pathlib import Path
    import re, numbers

    ALWAYS_DECIMAL = True  # pokaż co najmniej 1 miejsce po przecinku

    # 1) Ustal ile miejsc po przecinku powinna mieć KAŻDA kolumna numeryczna
    decimals_per_col = {}
    for col in df.columns:
        max_dec = 0
        all_num = True
        for v in df[col].tolist():
            if isinstance(v, numbers.Number) and not isinstance(v, bool):
                s = format(float(v), '.12g')              # obetnij ogonki
                if 'e' in s or 'E' in s:                  # uniknij notacji naukowej
                    s = f"{float(v):.12f}".rstrip('0').rstrip('.')
                dec = len(s.split('.', 1)[1]) if '.' in s else 0
                if dec > max_dec:
                    max_dec = dec
            else:
                all_num = False
        decimals_per_col[col] = max_dec if all_num else None
        if decimals_per_col[col] is not None and ALWAYS_DECIMAL and decimals_per_col[col] < 1:
            decimals_per_col[col] = 1

    def _esc_text(s: str) -> str:
        return (s.replace('&', r'\&').replace('%', r'\%').replace('$', r'\$')
                 .replace('#', r'\#').replace('_', r'\_').replace('{', r'\{')
                 .replace('}', r'\}').replace('~', r'\textasciitilde{}')
                 .replace('^', r'\textasciicircum{}'))

    def _fmt_cell(val, col_name):
        decs = decimals_per_col.get(col_name)
        if isinstance(val, numbers.Number) and not isinstance(val, bool) and decs is not None:
            s = f"{float(val):.{decs}f}"      # ZAOKRĄGL + DOPADUJ zerami
            s = s.replace('.', ',')           # europejski separator
            return _esc_text(s)
        else:
            return _esc_text(str(val))

    # 2) Składanie LaTeX-a (format i wcięcia jak w Twoim przykładzie)
    headers = [_esc_text(c) for c in df.columns]
    align = 'l' + 'r' * (len(headers) - 1)

    lines = []
    lines.append(r'\begin{table}[htbp]')
    lines.append(f'  \\caption[{_esc_text(short_caption)}]{{{_esc_text(long_caption)}}}')
    lines.append(f'  \\label{{{label}}}')
    lines.append(r'  \centering')
    lines.append(f'  \\begin{{tabular}}{{{align}}}')
    lines.append(r'    \toprule')
    lines.append('    ' + ' & '.join(headers) + r' \\')
    lines.append(r'    \midrule')
    for _, row in df.iterrows():
        row_cells = [_fmt_cell(v, col) for col, v in zip(df.columns, row.tolist())]
        lines.append('    ' + ' & '.join(row_cells) + r' \\')
    lines.append(r'    \bottomrule')
    lines.append(r'  \end{tabular}')
    lines.append(r'\end{table}')

    latex_text = '\n'.join(lines)

    # 3) Zapis do images/<label>.txt
    out_dir = Path('images')
    out_dir.mkdir(parents=True, exist_ok=True)
    safe_label = re.sub(r'[^A-Za-z0-9_-]+', '_', str(label))
    (out_dir / f'{safe_label}.txt').write_text(latex_text, encoding='utf-8')

    return latex_text


In [25]:
approach_name = 't2'
chapter_dir = "images\chapter03"

root_dir = f'.\\checkpoints\\modern\\stage_{approach_name}'
arch_dirs = glob.glob(os.path.join(root_dir, '*'))

def pareto_front(df: pd.DataFrame,
                 miou_col: str = "mIoU",
                 fps_col: str = "FPS",
                 eps: float = 1e-9) -> pd.DataFrame:

    d = df.dropna(subset=[miou_col, fps_col]).copy()

    d.sort_values([miou_col, fps_col], ascending=[False, False], inplace=True)

    best_fps = -np.inf
    keep = []
    for f in d[fps_col].to_numpy():
        if f >= best_fps - eps:
            keep.append(True)
            if f > best_fps:
                best_fps = f
        else:
            keep.append(False)

    front = d.loc[keep].copy()
    return front


<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\fhcar\AppData\Local\Temp\ipykernel_27892\4013914254.py:2: SyntaxWarning: invalid escape sequence '\c'
  chapter_dir = "images\chapter03"


In [26]:
arch = []
back = []
miou = []
std_5 = []
std_10 = []
fps = []

for arch_dir in arch_dirs:
    metrics_files = get_metric_files(arch_dir)
    for file in metrics_files:
        metrics = get_metrics(file[0])
        metadata = get_metadata(file[1])
        
        print(metrics.columns)
        arch.append(metadata.get("name"))
        back.append(metadata.get("encoder"))
        miou.append(metrics['val_miou_dataset'].tail(5).mean().round(4) * 100)
        std_5.append(metrics['val_miou_dataset'].tail(5).std().round(4) * 100)
        std_10.append(metrics['val_miou_dataset'].tail(10).std().round(4) * 100)
        fps.append(round(float(metadata.get("benchmark").get("fps")), 1))

df_table1 = pd.DataFrame({
    'Architektura': arch, 
    'Enkoder': back, 
    'FPS': fps,
    'mIoU [%]': miou, 
    'std_5': std_5, 
    'std_10': std_10
})

df_table1 = (
    df_table1.groupby("Architektura", group_keys=False)
      .apply(lambda g: pareto_front(g, "mIoU [%]", "FPS"))
).sort_values('mIoU [%]', ascending=False)

df_to_latex_table(
    df=df_table1, 
    short_caption='Tabela z podstawowymi wynikami optymalnych konfiguracji (Cityscapes)',
    long_caption='Tabela wyników optymalnych konfiguracji różnych architektur, uporządkowana malejąco według mIoU.',
    label='table:results1'
    )

Index(['epoch', 'f1_Bicycle 1', 'f1_Buildings', 'f1_Car 1',
       'f1_Grid structure', 'f1_Nature object', 'f1_Non-drivable street',
       'f1_Pedestrian 1', 'f1_Poles', 'f1_RD normal street', 'f1_Road blocks',
       'f1_Sidewalk', 'f1_Sky', 'f1_Small vehicles 1', 'f1_Traffic sign 1',
       'f1_Traffic signal 1', 'f1_Truck 1', 'f1_class_16', 'iou_Bicycle 1',
       'iou_Buildings', 'iou_Car 1', 'iou_Grid structure', 'iou_Nature object',
       'iou_Non-drivable street', 'iou_Pedestrian 1', 'iou_Poles',
       'iou_RD normal street', 'iou_Road blocks', 'iou_Sidewalk', 'iou_Sky',
       'iou_Small vehicles 1', 'iou_Traffic sign 1', 'iou_Traffic signal 1',
       'iou_Truck 1', 'iou_class_16', 'lr', 'step', 'val_acc_macro',
       'val_f1_macro', 'val_loss', 'val_memory_cpu', 'val_memory_gpu',
       'val_miou_dataset', 'val_miou_dataset_micro', 'val_miou_image',
       'val_prec_macro', 'val_rec_macro'],
      dtype='object')
Index(['epoch', 'f1_Bicycle 1', 'f1_Buildings', 'f1_Car 1'

C:\Users\fhcar\AppData\Local\Temp\ipykernel_27892\4192057887.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pareto_front(g, "mIoU [%]", "FPS"))


'\\begin{table}[htbp]\n  \\caption[Tabela z podstawowymi wynikami optymalnych konfiguracji (Cityscapes)]{Tabela wyników optymalnych konfiguracji różnych architektur, uporządkowana malejąco według mIoU.}\n  \\label{table:results1}\n  \\centering\n  \\begin{tabular}{lrrrrr}\n    \\toprule\n    Architektura & Enkoder & FPS & mIoU [\\%] & std\\_5 & std\\_10 \\\\\n    \\midrule\n    DeepLabV3Plus & efficientnet-b2 & 45,2 & 68,53 & 0,07 & 0,38 \\\\\n    DeepLabV3Plus & resnet34 & 65,8 & 66,04 & 0,12 & 0,10 \\\\\n    PAN & resnet101 & 30,5 & 66,03 & 0,19 & 0,37 \\\\\n    PAN & resnet50 & 40,6 & 64,24 & 0,21 & 0,27 \\\\\n    PAN & efficientnet-b2 & 51,5 & 63,19 & 0,13 & 0,30 \\\\\n    Linknet & mobileone\\_s4 & 14,2 & 62,77 & 0,35 & 0,78 \\\\\n    DeepLabV3Plus & resnet18 & 77,7 & 62,72 & 0,29 & 0,31 \\\\\n    Linknet & efficientnet-b4 & 31,5 & 62,47 & 0,13 & 0,65 \\\\\n    Linknet & mobileone\\_s2 & 39,7 & 62,39 & 0,51 & 0,72 \\\\\n    PAN & mobileone\\_s2 & 54,4 & 61,93 & 0,14 & 0,19 \\\\\n 

In [27]:
arch = []
back = []
miou = []
miou_small = []
miou_big = []
miou_critical = []
fps = []

small_c = ['iou_Pedestrian 1', 'iou_Traffic signal 1', 'iou_Traffic sign 1', 'iou_Poles']
big_c = ['iou_Buildings', 'iou_RD normal street', 'iou_Nature object', 'iou_Sky']
critical_c = ['iou_Car 1', 'iou_Pedestrian 1', 'iou_Bicycle 1']

for arch_dir in arch_dirs:
    metrics_files = get_metric_files(arch_dir)
    for file in metrics_files:
        metrics = get_metrics(file[0])
        metadata = get_metadata(file[1])

        arch.append(metadata.get("name"))
        back.append(metadata.get("encoder"))
        miou.append(metrics['val_miou_dataset'].tail(5).mean().round(4) * 100)
        miou_small.append(metrics[small_c].tail(5).mean().mean().round(4) * 100)
        miou_big.append(metrics[big_c].tail(5).mean().mean().round(4) * 100)
        miou_critical.append(metrics[critical_c].tail(5).mean().mean().round(4) * 100)
        fps.append(round(float(metadata.get("benchmark").get("fps")), 1))

df_table2 = pd.DataFrame({
    'Architektura': arch, 
    'Enkoder': back, 
    'FPS': fps,
    'mIoU': miou, 
    'mIoU_small': miou_small,
    'mIoU_big': miou_big,
    'mIoU_cri': miou_critical
})

df_table2 = (
    df_table2.groupby("Architektura", group_keys=False)
      .apply(lambda g: pareto_front(g, "mIoU", "FPS"))
).sort_values('mIoU', ascending=False)

df_to_latex_table(
    df=df_table2.drop(columns='FPS'), 
    short_caption='Tabela z wynikami różnych zestawów klas optymalnych konfiguracji (Cityscapes)',
    long_caption='Tabela wyników optymalnych konfiguracji różnych architektur, uporządkowana malejąco według mIoU.',
    label='table:results2'
    )

df_table2

C:\Users\fhcar\AppData\Local\Temp\ipykernel_27892\3701539965.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pareto_front(g, "mIoU", "FPS"))


,Architektura,Enkoder,FPS,mIoU,mIoU_small,mIoU_big,mIoU_cri
3,DeepLabV3Plus,efficientnet-b2,45.2,68.53,58.01,93.28,64.58
7,DeepLabV3Plus,resnet34,65.8,66.04,56.78,93.04,62.66
20,PAN,resnet101,30.5,66.03,54.10,92.47,63.79
16,PAN,resnet50,40.6,64.24,53.89,92.19,62.59
18,PAN,efficientnet-b2,51.5,63.19,50.65,92.17,58.79
15,Linknet,mobileone_s4,14.2,62.77,54.73,91.62,58.53
0,DeepLabV3Plus,resnet18,77.7,62.72,52.16,91.72,59.87
14,Linknet,efficientnet-b4,31.5,62.47,51.52,92.61,51.73
10,Linknet,mobileone_s2,39.7,62.39,55.74,91.40,60.87
21,PAN,mobileone_s2,54.4,61.93,52.08,91.84,60.35


In [29]:
arch = []
back = []
miou = []
miou_car = []
miou_person = []
miou_bicycle = []
miou_pole = []
miou_sign = []
fps = []

critical_c = ['iou_Car 1', 'iou_Pedestrian 1', 'iou_Bicycle 1']

for arch_dir in arch_dirs:
    metrics_files = get_metric_files(arch_dir)
    for file in metrics_files:
        metrics = get_metrics(file[0])
        metadata = get_metadata(file[1])

        arch.append(metadata.get("name"))
        back.append(metadata.get("encoder"))
        miou.append(metrics['val_miou_dataset'].tail(5).mean().round(4) * 100)
        miou_car.append(metrics['iou_Car 1'].tail(5).mean().round(4) * 100)
        miou_person.append(metrics['iou_Pedestrian 1'].tail(5).mean().round(4) * 100)
        miou_bicycle.append(metrics['iou_Bicycle 1'].tail(5).mean().round(4) * 100)
        miou_pole.append(metrics['iou_Poles'].tail(5).mean().round(4) * 100)
        miou_sign.append(metrics['iou_Traffic sign 1'].tail(5).mean().round(4) * 100)
        fps.append(round(float(metadata.get("benchmark").get("fps")), 1))

df_table2 = pd.DataFrame({
    'Architektura': arch, 
    'Enkoder': back, 
    'FPS': fps,
    'mIoU': miou, 
    'mIoU_car': miou_car,
    'mIoU_person': miou_person,
    'mIoU_bicycle': miou_bicycle,
    'mIoU_pole': miou_pole,
    'mIoU_sign': miou_sign
})

df_table2 = (
    df_table2.groupby("Architektura", group_keys=False)
      .apply(lambda g: pareto_front(g, "mIoU", "FPS"))
).sort_values('mIoU', ascending=False)

df_to_latex_table(
    df=df_table2.drop(columns='FPS'), 
    short_caption='Tabela z wynikami różnych klas dla optymalnych konfiguracji (Cityscapes)',
    long_caption='Tabela wyników optymalnych konfiguracji różnych architektur, uporządkowana malejąco według mIoU.',
    label='table:results4'
    )

df_table2

C:\Users\fhcar\AppData\Local\Temp\ipykernel_27892\2406302902.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pareto_front(g, "mIoU", "FPS"))


,Architektura,Enkoder,FPS,mIoU,mIoU_car,mIoU_person,mIoU_bicycle,mIoU_pole,mIoU_sign
3,DeepLabV3Plus,efficientnet-b2,45.2,68.53,91.52,55.58,46.65,48.66,74.26
7,DeepLabV3Plus,resnet34,65.8,66.04,91.10,53.98,42.89,46.67,73.42
20,PAN,resnet101,30.5,66.03,89.52,54.53,47.34,42.07,69.25
16,PAN,resnet50,40.6,64.24,87.49,54.84,45.45,42.18,69.05
18,PAN,efficientnet-b2,51.5,63.19,88.43,48.12,39.82,40.52,66.89
15,Linknet,mobileone_s4,14.2,62.77,88.15,48.08,39.36,48.72,70.18
0,DeepLabV3Plus,resnet18,77.7,62.72,89.51,50.02,40.07,42.48,69.62
14,Linknet,efficientnet-b4,31.5,62.47,89.83,34.55,30.80,45.40,73.87
10,Linknet,mobileone_s2,39.7,62.39,86.03,53.24,43.34,47.82,68.75
21,PAN,mobileone_s2,54.4,61.93,87.25,51.35,42.43,40.77,67.79
